In [1]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport GLS_Jan_weekend

No dicts found; please check load_dicts...


In [2]:
xi_list = GLS_Jan_weekend.xi_list
P = GLS_Jan_weekend.P
L = GLS_Jan_weekend.L  # dimension of xi

113

In [3]:
size(P, 1), size(P, 2)

(56,113)

In [4]:
using JuMP

mGLSJulia = Model()

@defVar(mGLSJulia, lam[1:size(P,1)] >= 0)

@defVar(mGLSJulia, p[1:size(P,1), 1:size(P,2)] >= 0)

for i = 1:size(P,1)
    for j = 1:size(P,2)
        if P[i,j] == 0
            @addConstraint(mGLSJulia, p[i,j] == 0)
        end
    end
end
            
for i = 1:size(P,1)
    @addNLConstraint(mGLSJulia, sum{p[i,j], j = 1:size(P,2)} == 1)
end

for l = 1:L
    @addNLConstraint(mGLSJulia, sum{p[i,l] * lam[i], i = 1:size(P,1)} == xi_list[l])
end
    
@setNLObjective(mGLSJulia, Min, sum{p[1,j], j = 1:size(P,2)})  # play no actual role, but could not use zero objective

solve(mGLSJulia)

 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in @defVar(::Symbol, ::Vararg{Any,N}) at /home/jzh/.julia/v0.5/JuMP/src/deprecated.jl:24
 in include_string(::String, ::String) at ./loading.jl:441
 in execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/jzh/.julia/v0.5/IJulia/src/execute_request.jl:169
 in eventloop(::ZMQ.Socket) at /home/jzh/.julia/v0.5/IJulia/src/eventloop.jl:8
 in (::IJulia.##9#15)() at ./task.jl:360
while loading In[4], in expression starting on line 5



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    25199
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:    18984

Total number of variables............................:     6384
                     variables with only lower bounds:     6384
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

:Optimal

In [5]:
getValue(lam)

 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in getValue(::Array{JuMP.Variable,1}, ::Vararg{Array{JuMP.Variable,1},N}) at ./deprecated.jl:30
 in include_string(::String, ::String) at ./loading.jl:441
 in execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/jzh/.julia/v0.5/IJulia/src/execute_request.jl:169
 in eventloop(::ZMQ.Socket) at /home/jzh/.julia/v0.5/IJulia/src/eventloop.jl:8
 in (::IJulia.##9#15)() at ./task.jl:360
while loading In[5], in expression starting on line 1


56-element Array{Float64,1}:
  496.376
  990.782
  936.718
  587.588
  495.053
  383.247
  312.39 
  590.195
  172.335
 1533.66 
  215.756
  140.415
  303.868
    ⋮    
  591.65 
  597.028
  443.35 
  580.311
  280.799
  350.688
  256.934
  379.701
 1802.07 
  193.277
  409.766
  295.302

In [6]:
getObjectiveValue(mGLSJulia)

1.0000000000001825

In [7]:
GLS_Jan_weekend.saveDemandVec(getValue(lam))